In [28]:
import os
import datetime
import nest_asyncio
from dotenv import load_dotenv
from graphiti_core import Graphiti
from graphiti_core.llm_client.gemini_client import GeminiClient, LLMConfig
from graphiti_core.embedder.gemini import GeminiEmbedder, GeminiEmbedderConfig
from graphiti_core.utils.bulk_utils import EpisodeType
nest_asyncio.apply()
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")


if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not set in .env")

if not NEO4J_PASSWORD:
    raise ValueError("NEO4J_PASSWORD not set in .env")

async def simple_graphiti_gemini_example():
    """
    Simple example showing how to use Graphiti with Gemini
    """

    gemini_llm_client = GeminiClient(
        config=LLMConfig(
            api_key=GOOGLE_API_KEY,
            model="gemini-2.5-flash"
        )
    )

    gemini_embedder = GeminiEmbedder(
        config=GeminiEmbedderConfig(
            api_key=GOOGLE_API_KEY,
            embedding_model="text-embedding-004"
        )
    )

    graphiti = Graphiti(
        NEO4J_URI,
        NEO4J_USERNAME,
        NEO4J_PASSWORD,
        llm_client=gemini_llm_client,
        embedder=gemini_embedder
    )

    print(" Graphiti initialized with Gemini!")

    await graphiti.build_indices_and_constraints()
    print(" Database schema initialized!")

    episodes = [
        "user033 is Omani always buy Apple brand less than 500 OMR.",
        "user005 always ask about latest new products from Apple, and ordered 3 Apple smart watches.",
        "user001 like same brand as user033. He makes new order every friday.",
        "user005 recently looking for a new smart watches from Samsung and no more likes Apple brand.",
        "user001 preferences are iphone 15 pro, Apple Watch Ultra 2, and Airpods 3X pro"
    ]

    print("\n Adding episodes to knowledge graph...")
    for i, episode in enumerate(episodes, 1):
        await graphiti.add_episode(
            name=f"episode_{i}",
            episode_body=episode,
            source=EpisodeType.text,
            source_description="User preferences",
            reference_time=datetime.datetime.now()
        )
        print(f" Added episode {i}")

    print("\n Querying the knowledge graph...")

    queries = [
        "what's the preferred brand for user005?"
    ]

    for query in queries:
        results = await graphiti.search(query=query, num_results=3)
        print(f"\n Results for query: {query}")
        for i, result in enumerate(results, 1):
            embedding = result.attributes['fact_embedding'] if result.attributes and 'fact_embedding' in result.attributes else None

            if embedding:
                print(f"   {i}. {result.fact}")
                print(f"       Embedding: {len(embedding)} dimensions")
            else:
                print(f"   {i}. {result.fact}")
                print(f"       No embedding found")

await simple_graphiti_gemini_example()

 Graphiti initialized with Gemini!
 Database schema initialized!

 Adding episodes to knowledge graph...
 Added episode 1
 Added episode 2
 Added episode 3
 Added episode 4
 Added episode 5

 Querying the knowledge graph...

 Results for query: what's the preferred brand for user005?
   1. user001 like same brand as user033
       Embedding: 768 dimensions
   2. user005 no more likes Apple brand
       Embedding: 768 dimensions
   3. user005 no more likes Apple brand
       Embedding: 768 dimensions


In [30]:
import os
import nest_asyncio
from dotenv import load_dotenv
from graphiti_core import Graphiti
from graphiti_core.llm_client.gemini_client import GeminiClient, LLMConfig
from graphiti_core.embedder.gemini import GeminiEmbedder, GeminiEmbedderConfig

nest_asyncio.apply()
load_dotenv()


GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

# Initialize Graphiti
gemini_llm_client = GeminiClient(
    config=LLMConfig(
        api_key=GOOGLE_API_KEY,
        model="gemini-2.5-flash"
    )
)

gemini_embedder = GeminiEmbedder(
    config=GeminiEmbedderConfig(
        api_key=GOOGLE_API_KEY,
        embedding_model="text-embedding-004"
    )
)

graphiti = Graphiti(
    NEO4J_URI,
    NEO4J_USERNAME,
    NEO4J_PASSWORD,
    llm_client=gemini_llm_client,
    embedder=gemini_embedder
)

results = await graphiti.search(query="what's the preferred brand for user005?", num_results=3)

for i, result in enumerate(results, 1):
    embedding = result.attributes['fact_embedding'] if result.attributes and 'fact_embedding' in result.attributes else None
    print(f"{i}. {result} - Embedding: {len(embedding) if embedding else 'None'}")

1. uuid='3ee2540c-a83d-4b98-98a3-eac9593de1b7' group_id='' source_node_uuid='a6641bf9-66da-4dfb-b707-b3ffd2ece7f6' target_node_uuid='f779a360-c80a-4e24-a603-7b9f3892fb06' created_at=datetime.datetime(2025, 7, 8, 8, 37, 41, 735744, tzinfo=<UTC>) name='LIKES_SAME_BRAND_AS' fact='user001 like same brand as user033' fact_embedding=None episodes=['6a9449fd-c927-49ac-ba9b-400c4331f46f', '5a63aca6-d924-44a4-b88d-85a542d1c209'] expired_at=None valid_at=datetime.datetime(2025, 7, 8, 12, 37, 36, 433068, tzinfo=<UTC>) invalid_at=None attributes={'fact_embedding': [-0.01990159973502159, -0.02932509034872055, -0.03530290350317955, -0.0033561475574970245, 0.06710278987884521, -0.0070217750035226345, 0.03053075447678566, 0.004622977692633867, -0.04125690460205078, -0.006299776490777731, 0.014705303125083447, 0.03365815430879593, 0.028191694989800453, 0.01140576135367155, -0.015037198550999165, -0.03752588480710983, 0.0508541539311409, 0.08353683352470398, -0.0751558318734169, -0.003845051396638155, -

In [13]:
try:
    test_embedding = await gemini_embedder.create("test text")
    print(f"Using 'create' method - Embedding type: {type(test_embedding)}")
    print(f"Embedding length: {len(test_embedding) if hasattr(test_embedding, '__len__') else 'No length'}")
    if hasattr(test_embedding, '__len__') and len(test_embedding) > 0:
        print(f"First 5 values: {test_embedding[:5]}")
except Exception as e:
    print(f"Error with 'create' method: {e}")

try:
    test_batch = await gemini_embedder.create_batch(["test text"])
    print(f"Using 'create_batch' method - Result type: {type(test_batch)}")
    if isinstance(test_batch, list) and len(test_batch) > 0:
        print(f"First embedding length: {len(test_batch[0]) if hasattr(test_batch[0], '__len__') else 'No length'}")
        if hasattr(test_batch[0], '__len__') and len(test_batch[0]) > 0:
            print(f"First 5 values: {test_batch[0][:5]}")
except Exception as e:
    print(f"Error with 'create_batch' method: {e}")

Using 'create' method - Embedding type: <class 'list'>
Embedding length: 768
First 5 values: [-0.021624343, 0.041636877, -0.046839744, 0.019721426, 0.06168559]
Using 'create_batch' method - Result type: <class 'list'>
First embedding length: 768
First 5 values: [-0.021624343, 0.041636877, -0.046839744, 0.019721426, 0.06168559]


In [12]:
print("Available methods on GeminiEmbedder:")
print([method for method in dir(gemini_embedder) if not method.startswith('_')])

Available methods on GeminiEmbedder:
['client', 'config', 'create', 'create_batch']


In [15]:
async def debug_graphiti_embedding_flow():
    print(f"Graphiti embedder: {graphiti.embedder}")
    print(f"Embedder is same object: {graphiti.embedder is gemini_embedder}")

    recent_results = await graphiti.search(query="user preferences", num_results=5)

    for i, result in enumerate(recent_results):
        print(f"\n--- Result {i + 1} ---")
        print(f"Fact: {result.fact}")
        print(f"Main fact_embedding is None: {result.fact_embedding is None}")

        if result.attributes and 'fact_embedding' in result.attributes:
            attr_embedding = result.attributes['fact_embedding']
            print(f"Attributes embedding length: {len(attr_embedding) if attr_embedding else 'None'}")
            print(f"Attributes embedding type: {type(attr_embedding)}")
            if attr_embedding and len(attr_embedding) > 0:
                print(f"First 3 values: {attr_embedding[:3]}")


await debug_graphiti_embedding_flow()


Graphiti embedder: <graphiti_core.embedder.gemini.GeminiEmbedder object at 0x1184596a0>
Embedder is same object: True

--- Result 1 ---
Fact: user001 preferences are iphone 15 pro
Main fact_embedding is None: True
Attributes embedding length: 768
Attributes embedding type: <class 'list'>
First 3 values: [-0.015845874324440956, -0.0290859192609787, -0.07403789460659027]

--- Result 2 ---
Fact: user001 preferences are Apple Watch Ultra 2
Main fact_embedding is None: True
Attributes embedding length: 768
Attributes embedding type: <class 'list'>
First 3 values: [-0.006810247432440519, -0.0012139372993260622, -0.06753826886415482]

--- Result 3 ---
Fact: user001 preferences are Airpods 3X pro
Main fact_embedding is None: True
Attributes embedding length: 768
Attributes embedding type: <class 'list'>
First 3 values: [0.013378829695284367, -0.034592896699905396, -0.07346352189779282]

--- Result 4 ---
Fact: user001 like same brand as user033
Main fact_embedding is None: True
Attributes embed

In [16]:
import pkg_resources
try:
    version = pkg_resources.get_distribution("graphiti-core").version
    print(f"Current graphiti-core version: {version}")
except:
    print("Could not determine version")

import subprocess
result = subprocess.run(["pip", "list", "--outdated", "--format=json"], capture_output=True, text=True)
print("Check if graphiti-core appears in outdated packages")

Current graphiti-core version: 0.14.0
Check if graphiti-core appears in outdated packages


test graphiti main issue with simple test case


In [17]:
async def test_minimal_episode():
    """
    Test with the absolute minimal example to prove the issue in the graphiti's storage
    """

    # Test 1: Add one simple episode
    await graphiti.add_episode(
        name="test_embedding_location",
        episode_body="TestUser likes TestBrand products",
        source=EpisodeType.text,
        source_description="Minimal test",
        reference_time=datetime.datetime.now()
    )

    # Test 2: Immediately search for it
    results = await graphiti.search(query="TestUser TestBrand", num_results=1)

    if results:
        result = results[0]
        print("=== MINIMAL TEST RESULT ===")
        print(f"Fact: {result.fact}")
        print(f"Main fact_embedding is None: {result.fact_embedding is None}")
        print(f"Has attributes: {bool(result.attributes)}")

        if result.attributes and 'fact_embedding' in result.attributes:
            attr_emb = result.attributes['fact_embedding']
            print(f"Attributes embedding exists: {attr_emb is not None}")
            print(f"Attributes embedding length: {len(attr_emb) if attr_emb else 'None'}")
            print(" CONFIRMED: Embedding in wrong location!")
        else:
            print(" No embedding found anywhere!")
    else:
        print(" No results found")

await test_minimal_episode()

=== MINIMAL TEST RESULT ===
Fact: TestUser likes TestBrand products
Main fact_embedding is None: True
Has attributes: True
Attributes embedding exists: True
Attributes embedding length: 768
 CONFIRMED: Embedding in wrong location!


In [18]:
print("=== CONFIGURATION CHECK ===")
print(f"Embedder model: {gemini_embedder.config.embedding_model}")
print(f"LLM model: {gemini_llm_client.config.model}")

print(f"NEO4J_DATABASE: {NEO4J_DATABASE}")

# Test direct embedding vs Graphiti embedding
direct_embedding = await gemini_embedder.create("TestUser likes TestBrand products")
print(f"Direct embedding length: {len(direct_embedding)}")

# Check if there are any Graphiti-specific settings
graphiti_attrs = [attr for attr in dir(graphiti) if not attr.startswith('_')]
embedding_related = [attr for attr in graphiti_attrs if 'embed' in attr.lower()]
print(f"Graphiti embedding-related attributes: {embedding_related}")

=== CONFIGURATION CHECK ===
Embedder model: text-embedding-004
LLM model: gemini-2.5-flash
NEO4J_DATABASE: gemini
Direct embedding length: 768
Graphiti embedding-related attributes: ['embedder']


In [4]:
# full_agent_flow.py
# Complete memory-aware tool calling flow using Neo4j for structured logging

from neo4j import GraphDatabase
import uuid
from datetime import datetime
import time
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

class GraphMemory:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def log_tool_call(self, session_id, tool_name, args, result):
        call_id = str(uuid.uuid4())
        timestamp = datetime.utcnow().isoformat()
        with self.driver.session() as session:  # Removed database=NEO4J_DATABASE
            session.write_transaction(self._create_tool_memory,
                                      session_id, call_id, timestamp,
                                      tool_name, args, result)

    @staticmethod
    def _create_tool_memory(tx, session_id, call_id, timestamp, tool_name, args, result):
        query = """
        MERGE (s:Session {id: $session_id})
        CREATE (c:ToolCall {
            id: $call_id,
            name: $tool_name,
            args: $args,
            timestamp: $timestamp
        })
        CREATE (r:ToolResponse {
            id: $call_id,
            result: $result,
            timestamp: $timestamp
        })
        MERGE (s)-[:HAS_CALL]->(c)
        MERGE (c)-[:GOT_RESULT]->(r)
        """
        tx.run(query, session_id=session_id, call_id=call_id,
               timestamp=timestamp, tool_name=tool_name,
               args=args, result=result)

class Agent:
    def __init__(self, memory: GraphMemory, session_id: str):
        self.memory = memory
        self.session_id = session_id

    def call_tool(self, tool_name, args):
        print(f"Calling tool: {tool_name} with args: {args}")
        # Simulated tool behavior
        if tool_name == "search_products":
            if args['filters']['color'] == "red":
                result = {"total_found": 0, "products": []}
            else:
                result = {"total_found": 1, "products": [{"name": "iPhone 15 Pro", "color": "black", "price": 395}]}
        elif tool_name == "place_order":
            result = {"status": "success", "order_number": "ORD9981"}
        else:
            result = {"status": "unknown tool"}

        # Log to memory
        self.memory.log_tool_call(self.session_id, tool_name, args, result)
        return result

    def handle_user_request(self):
        print("User wants to buy an iPhone 15 Pro under 400 OMR")

        # Attempt 1: red iPhone
        args1 = {"text": "iphone 15 pro", "filters": {"color": "red", "price_range": {"max": 400}}}
        result1 = self.call_tool("search_products", args1)

        if result1['total_found'] == 0:
            print("No products found in red. Retrying with black...")
            time.sleep(1)  # simulate thinking time
            args2 = {"text": "iphone 15 pro", "filters": {"color": "black", "price_range": {"max": 400}}}
            result2 = self.call_tool("search_products", args2)

            if result2['total_found'] > 0:
                product = result2['products'][0]
                print(f"Found product: {product['name']} for {product['price']} OMR")
                order_args = {"product_id": "abc123", "user_id": "U456"}
                result3 = self.call_tool("place_order", order_args)

                if result3['status'] == "success":
                    print(f"✅ Order placed! Order number: {result3['order_number']}")
                else:
                    print("❌ Failed to place order")
        else:
            print("Found product in first try!")

if __name__ == "__main__":
    memory = GraphMemory(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
    session_id = "chat-001"

    agent = Agent(memory, session_id)
    agent.handle_user_request()

    memory.close()


User wants to buy an iPhone 15 Pro under 400 OMR
Calling tool: search_products with args: {'text': 'iphone 15 pro', 'filters': {'color': 'red', 'price_range': {'max': 400}}}


/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_3235/644623141.py:28: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()
/var/folders/fd/sjwsgyf54kb5m8t323fr7qsh0000gn/T/ipykernel_3235/644623141.py:30: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_tool_memory,


CypherTypeError: {code: Neo.ClientError.Statement.TypeError} {message: Property values can only be of primitive types or arrays thereof. Encountered: Map{text -> String("iphone 15 pro"), filters -> Map{color -> String("red"), price_range -> Map{max -> Long(400)}}}.}